In [ ]:
!pip install clickhouse-connect sentence_transformers transformers clickhouse_driver spacy
!python -m spacy download ru_core_news_sm


In [ ]:
#делаем все необходимые импорты
import numpy as np
import torch

from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import set_seed
from sklearn.metrics.pairwise import cosine_similarity
import clickhouse_connect
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,AutoModelForCausalLM,AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer, util
from utils.generators import SpellChecker, ToxicityClassifier

import clickhouse_connect

from scipy.spatial.distance import cdist
import spacy
import re

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

set_seed(42)

nltk.download('punkt')
torch.cuda.is_available()


In [1]:
# Загрузка модели для эмбеддинга
embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")

# Загрузка токенизатора и модели T5
t5_tokenizer = AutoTokenizer.from_pretrained("ai-forever/FRED-T5-1.7B")
t5_model = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/FRED-T5-1.7B")

#Загрузим проверщика орфографии 
spelchecker_model_name = 'UrukHan/t5-russian-spell'

spell_checker_tokenizer = T5TokenizerFast.from_pretrained(spelchecker_model_name)
spell_checker_model = AutoModelForSeq2SeqLM.from_pretrained(spelchecker_model_name).to(device).eval()

#Проверка на токсичность вопросов
toxicity_detection_model_name = 'cointegrated/rubert-tiny-toxicity'
toxicity_detection_tokenizer = AutoTokenizer.from_pretrained(toxicity_detection_model_name)
toxicity_detection_model = AutoModelForSequenceClassification.from_pretrained(toxicity_detection_model_name).to(device).eval()

NameError: name 'SentenceTransformer' is not defined

In [ ]:
#подключаемся к нашей баззе данных
client = clickhouse_connect.get_client(
    host='x2ar8i584r.europe-west4.gcp.clickhouse.cloud',
    port=8443,
    username='default',
    password='Avk7dO5kAfK_s'  # Замените на ваш реальный пароль
)

In [ ]:
#получаем и предобрабатываем данные из базы данных
result = client.query('SELECT text,url from "document"')
docs_with_links = [
#     {'text': 'Текст документа 1', 'link': 'https://ссылка_на_документ_1'},
#     {'text': 'Текст документа 2', 'link': 'https://ссылка_на_документ_2'},
]


In [ ]:
#заполняем наш список текстами
for row in result.result_rows:
    # Удаляем специальные символы, кроме букв, цифр, пробелов, точек и запятых, заменяем серии пробелов на одинарные
    redacted_text = re.sub(r'[^\w\s\.,]+', '', row[0])  # Удаляем нежелательные символы
    redacted_text = re.sub(r'\s+', ' ', redacted_text)  # Заменяем серии пробелов на одинарные пробелы
    redacted_text = redacted_text.replace('\n', ' ').replace('\r', ' ').strip()  # Удаляем переносы строк и возвраты каретки, если они есть
    
    docs_with_links.append({
        "text": redacted_text,
        "url": row[1]
    })


In [ ]:
len(docs_with_links)

# Ниже закоментирован код деления данных на чанки(батчи), части текстов, которые в будущем и будут являтся контекстом для генерации модели. Закуоментирован он, т.к он выполняет запаолнение базы данных и не нужен нам каждый раз

In [ ]:
# #описиываем  деление текстов на чанки(подтексты)
# nlp = spacy.load("ru_core_news_sm")
# def create_chunks_by_sentences(text, sentences_per_chunk=2):
#     doc = nlp(text)
#     chunks = []
#     current_chunk = []
#     1
#     for sent in doc.sents:
#         current_chunk.append(sent.text.strip())
        
#         # Если в текущем чанке накопилось 2 предложения, сохраняем его и начинаем новый
#         if len(current_chunk) == sentences_per_chunk:
#             chunks.append(" ".join(current_chunk))
#             current_chunk = []
    
#     # Добавляем последний чанк, если он содержит предложения
#     if current_chunk:
#         chunks.append(" ".join(current_chunk))
        
#     return chunks

# def get_chunk_embeddings(data):
#     return embedding_model.encode(data)

In [ ]:
# chunk_id = 0  # Начальный ID для чанков

# for doc in docs_with_links[:3]:
#     text_chunks = create_chunks_by_sentences(doc['text'])
    
#     for chunk_text in text_chunks:
#         # Получаем эмбеддинг для чанка текста
#         embedding = get_chunk_embeddings(chunk_text)  # Это должен быть список float чисел
        
#         # Преобразуем список в строку в формате ClickHouse Array
#         embedding_str = '[' + ','.join(map(str, embedding)) + ']'
        
#         # Экранирование кавычек в тексте для SQL запроса
#         chunk_text_escaped = chunk_text.replace("'", "''")
        
#         # Формируем SQL запрос для вставки текста и эмбеддинга как массива
#         sql = f"INSERT INTO text_embeddings (chunk_id, chunk_text, embedding, source_url) VALUES ({chunk_id}, '{chunk_text_escaped}', {embedding_str}, '{doc['url']}')"
        
#         # Выполнение запроса
#         try:
#             client.query(sql)
#         except Exception as e:
#             print(f"Ошибка при вставке данных: {e}")
        
#         chunk_id += 1


# Ниже представлена реализация RAG(Retrieval Augmented Generation). Сначала мы находим n наиболее релевантных к нашему запросов чанков, объединяем их в 1 контекст и передаём модели для генерации ответа на его основе

In [ ]:
#метод извлекающий чанки и их эмбединги из баззы данных
def fetch_embeddings_from_db():
    query_result = client.query('SELECT chunk_text, embedding,source_url FROM text_embeddings')
    chunks = []
    embeddings = []
    urls = []
    for row in query_result.result_rows:
        chunk_text, embedding,url = row  # предполагаем, что embedding уже в нужном формате
        chunks.append(chunk_text)
        urls.append(url)
        # Преобразуем список в numpy массив, если он уже не в таком формате
        embedding_array = np.array(embedding, dtype=np.float32)
        
        # Добавляем эмбеддинг в список embeddings
        embeddings.append(embedding_array)
    
    
    return chunks, urls, np.array(embeddings)


In [ ]:
#получаем чанки и их эмбединги, а так же ссылки на документы, откуда чанк был взят
chunks, urls, chunk_embeddings = fetch_embeddings_from_db()

In [ ]:
#имеем 15 тысяч чанков и 15 тысячей эмбедингов
len(chunks),chunk_embeddings.shape

# RETRIEVING + CONETXT GETTING

In [ ]:
spell_checker = SpellChecker(spell_checker_tokenizer, spell_checker_model)
toxicity_detector = ToxicityClassifier(toxicity_detection_tokenizer, toxicity_detection_model)

In [ ]:
def retrieve_top_k_documents(query, top_k):
    query_embedding = embedding_model.encode([query])[0]
    cos_similarities = cosine_similarity(query_embedding.reshape(1, -1), np.array(chunk_embeddings))
    sorted_indices = np.argsort(cos_similarities[0])[::-1]

    unique_chunks = set()
    top_chunks = []
    top_indices = []
    for idx in sorted_indices:
        if len(top_chunks) >= top_k:
            break
        chunk = chunks[idx]
        if chunk not in unique_chunks:
            unique_chunks.add(chunk)
            top_chunks.append(chunk)
            top_indices.append(idx)

    return top_chunks, top_indices

In [ ]:
def get_context_from_query(query):
    top_chunks, top_k_indices = retrieve_top_k_documents(query, top_k=4)
    return "\n\n".join([f"{chunk}\nИсточник: {urls[top_k_indices[i]]}" for i, chunk in enumerate(top_chunks)])
    

    

# Применяем RAG и скармливаем полученный и обработанный контекст модели для генерации

In [ ]:
def get_response(prompt):
    input_ids = t5_tokenizer(prompt, return_tensors="pt").input_ids
    outputs = t5_model.generate(
    input_ids, 
    max_length=512, 
    num_beams=5, 
    early_stopping=True,
    do_sample=True, 
    top_k=50, 
    top_p=0.95, 
    repetition_penalty=2.5
)

    answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


In [ ]:
#Задаёте интерсующий Вас вопрос
query = input()
if toxicity_detector(query):
    print("Грубый вопрос")
corrected_query = spell_checker(query)
#для вашего вопроса автоматически формируется контекст
context = get_context_from_query(corrected_query)
#контекст + вопрос формируют промт для модели
prompt = f"""
Question: {corrected_query}

Context : {context}

"""


# Ответ модели:

In [ ]:
print(get_response(prompt))